In [1]:
"""Trains a convolutional neural network on the MNIST dataset, then attacks it with the FGSM attack."""
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

2024-02-13 19:04:41.262259: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 19:04:41.264895: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 19:04:41.295401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 19:04:41.295458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 19:04:41.296720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import numpy as np

In [5]:
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Read MNIST dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("mnist"))

In [7]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=5, batch_size=128)

2024-02-13 19:04:48.913046: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-13 19:04:48.975826: W tensorflow/c/c_api.cc:305] Operation '{name:'conv2d/bias/Assign' id:20 op device:{requested: '', assigned: ''} def:{{{node conv2d/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d/bias, conv2d/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [10]:
# Evaluate the classifier on the test set
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-02-13 19:07:38.107534: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_1/Softmax' id:121 op device:{requested: '', assigned: ''} def:{{{node dense_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.



Test accuracy: 99.14%


In [11]:
x_test_subset = x_test[:500]
y_test_subset = y_test[:500]

In [12]:
preds = np.argmax(classifier.predict(x_test_subset), axis=1)
acc = np.sum(preds == np.argmax(y_test_subset, axis=1)) / y_test_subset.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))


Test accuracy: 99.60%


In [14]:
jsma_attack = SaliencyMapMethod(classifier=classifier)

target = np.zeros((y_test_subset.shape[0], y_test_subset.shape[1]), dtype=np.float32)
target[:, 3] = 1  # This sets "3" as the target class for all examples

# Generate targeted adversarial examples with JSMA
x_test_adv_jsma = jsma_attack.generate(x=x_test_subset, y=target)

JSMA: 100%|██████████| 500/500 [08:55<00:00,  1.07s/it]


In [15]:
# Evaluate the classifier on the JSMA adversarial examples
preds_jsma = np.argmax(classifier.predict(x_test_adv_jsma), axis=1)
acc_jsma = np.sum(preds_jsma == np.argmax(y_test_subset, axis=1)) / y_test_subset.shape[0]
print("Test accuracy on adversarial samples (JSMA): {:.2f}%".format(acc_jsma * 100))

Test accuracy on adversarial samples (JSMA): 9.00%


In [16]:
from sklearn.metrics import precision_recall_fscore_support
y_test_subset = np.argmax(y_test_subset, axis=1)
precision, recall, fscore, support = precision_recall_fscore_support(y_test_subset, preds_jsma)

# Calculate micro-averaged precision, recall, and F1 score
precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
    y_test_subset, preds_jsma, average='micro')

# Calculate macro-averaged precision, recall, and F1 score
precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
    y_test_subset, preds_jsma, average='macro')

# Print the micro-averaged values
print(f"Micro-averaged precision: {precision_micro:.2f}")
print(f"Micro-averaged recall: {recall_micro:.2f}")
print(f"Micro-averaged F1 score: {fscore_micro:.2f}")

# Print the macro-averaged values
print(f"Macro-averaged precision: {precision_macro:.2f}")
print(f"Macro-averaged recall: {recall_macro:.2f}")
print(f"Macro-averaged F1 score: {fscore_macro:.2f}")

Micro-averaged precision: 0.09
Micro-averaged recall: 0.09
Micro-averaged F1 score: 0.09
Macro-averaged precision: 0.01
Macro-averaged recall: 0.10
Macro-averaged F1 score: 0.02


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
